# 네이버 영화 리뷰 스크래핑

In [11]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd

def get_movie_reviews(mcode, page_num=10):

    movie_review_df = pd.DataFrame(columns = ('Title', 'Score', 'Review'))
    url = "https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=" + str(mcode) +"&target=after"
    idx = 0 
        
    for _ in range(0, page_num):
        movie_page = urllib.request.urlopen(url).read()
        movie_page_soup = BeautifulSoup(movie_page, 'html.parser')
        
        review_list = movie_page_soup.find_all('td', {'class' : 'title'})
        for review in review_list:
            title = review.find('a', {'class' : 'movie color_b'}).get_text()
            score = review.find('em').get_text()
            review_text = review.find('a', {'class' : 'report'}).get('onclick').split(',')[2]
            
            #old_content > table > tbody > tr:nth-child(1) > td.title > a.report
            
            movie_review_df.loc[idx] = [title, score, review_text]
            idx += 1
            print("#", end="")
            
        try:        
            url = 'https://movie.naver.com' + movie_page_soup.find('a', {'class': 'pg_next'}).get('href')
        except:
            break
            
    return movie_review_df

In [13]:
movie_review_df = get_movie_reviews(150637, 1)
movie_review_df

##########

,Title,Score,Review
0,남한산성,10,'임금이 무능한 신하와 만나고
1,남한산성,10,'진득한 웰메이드 사극입니다. 자극적이진 않아 자칫 지루하게 느낄 수 있지만 몰입...
2,남한산성,10,'그냥 그럭저럭 보앗습니다좋아하는 배우들이 많이나와서 좋았어요'
3,남한산성,10,'평점 ㅈ박았네 ㅋㅋㅋ 하여튼 우리나라 수준 ㅋㅋㅋ'
4,남한산성,10,'정말 오랜만에 집중해서 영화를 봤습니다'
5,남한산성,10,'현실을 이성적으로 판단하지 못하고 감정적으로 대처하면 개박살이 나고서야 강제로 ...
6,남한산성,10,'펜트하우스 같은 저질 싸구려 드라마에 찌들어있는 사람들한테는 지루하겠지.'
7,남한산성,10,''
8,남한산성,10,'반도 역사상 가장 형편 없었던 국가 조선에 대해 이렇게나 진지하게 비판한 영화가...
9,남한산성,9,'평점수준보니 국사시간에 졸던 놈들이 많네'


## 현재상영작 영화 리뷰 스크래핑

In [16]:
url = "https://movie.naver.com/movie/point/af/list.naver"
naver_movie = urllib.request.urlopen(url).read()
soup = BeautifulSoup(naver_movie, 'html.parser')
select = soup.find('select', {'id': 'current_movie'})
movies = select.find_all('option')

movies_dict = {}
for movie in movies[1:]:
    movies_dict[movie.get('value')] = movie.get_text
    
movie_review_df = pd.DataFrame(columns={"Title", "Score", "Review"})
for mcode in movies_dict:
    df = get_movie_reviews(mcode, 1)
    movie_review_df = pd.concat([movie_review_df, df])
    
movie_review_df

########################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

,Score,Review,Title
0,9,'아이들이 보통은 극장을 무서워하거나 지루해하는데 꼼짝 않고 봐서 좋았다. 남매 ...,매직 스쿨: 초보마법사 아우라의 대모험
1,10,'조카들과 함께 보았는데 잼있게 봤습니다~!흥미진진한 스토리. 마법. Ost도 ...,매직 스쿨: 초보마법사 아우라의 대모험
2,10,'남자 아이들이 좋아해요!',매직 스쿨: 초보마법사 아우라의 대모험
3,1,'이거 언제만든건지 CG 강대국 미국에서 나온게 맞는건가...',매직 스쿨: 초보마법사 아우라의 대모험
4,10,'우리 딸이 너무 좋아해서 영화 보고 나서도 계속 이 얘기만 하네요~ 최고의 영화...,매직 스쿨: 초보마법사 아우라의 대모험
...,...,...,...
5,9,'100프로 이해가 되진 않지만,나쁜 피
6,10,'',나쁜 피
7,10,'갑자기 생각난 영화.ㅋㅋ 예전에 보았던 포스터가 망각속에 잊혀졌다가 다시 기억났...,나쁜 피
8,10,'젊음을 수혈해주는 영화',나쁜 피
